In [14]:
import pandas as pd
import numpy as np
from scipy import sparse
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import decomposition, datasets, model_selection, preprocessing, metrics
import src.rainbow
import os
import glob
import pickle

In [15]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
df_big = pickle.load(open('./pickles/df.p','rb'))

In [3]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 831284 entries, 5 to 2824
Data columns (total 12 columns):
date           831284 non-null datetime64[ns]
gross+         831284 non-null float64
gross-         831284 non-null float64
items          831284 non-null object
net            831284 non-null float64
oper           831284 non-null int64
term           831284 non-null int64
time           831284 non-null object
trans          831284 non-null int64
type           831284 non-null object
day_of_week    831284 non-null object
month          831284 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(4), object(4)
memory usage: 82.4+ MB


In [16]:
def get_items(df,most_common=10,least_common=5):
    df['total_of_items'] = 0
    stoppers = ['BAG CREDIT','SF Bag Charge','Gift Card Reload','8 OZ BIO TUB t3', '16OZ BIO TUB t4',
                 '32OZ BIO TUB t5','BOTTLE DEPOSIT','6PACK BEER SMALL C','PAID IN','Gift Card Sale','PACKAGED FOOD', ]  
    '''build a dictionary where the keys are the words
    in the dataframe items column'''
    
    items=[]
    item_dict = defaultdict(int)
    basket_counts=[]
    
    for basket in df['items']:
        basket_counts.append(len(basket))
        for item in basket:
            
            items.append(item[1])
            item_dict[item[1]] += 1
    
    items_set=set(items)

    
    '''add the most common words to the stopwords list'''
    stopwords=list([i[0] for i in Counter(item_dict).most_common(most_common)])
    
    for s in stoppers:
        stopwords.append(s)
        
    '''add items containing "CRV" to the stopwords list'''
    for item in items_set:
        if "crv" in item.lower():
            stopwords.append(item)
    
    '''add the least common words to the stopwords list'''
    for key,value in item_dict.items():
        if value < least_common:
            stopwords.append(key)
    print(type(stopwords) )  
    stopwords = set(stopwords)
    
    '''iterate through the baskets and add items to items_set
    if not in stopwords (too common or too uncommon)'''
    for stops in stopwords:
        try:
            items_set.remove(stops)
        except KeyError as k:
            continue

    return items_set,stopwords, item_dict, basket_counts



In [17]:
def build_matrix(df,items_set,stopwords,basket_counts):
    df['item_count']=basket_counts
    item_matrix = np.zeros((df.shape[0],len(items_set)))
    df_items= pd.DataFrame(item_matrix,columns=items_set)
    df=df.reset_index()
    df.pop('index')
    col_index_dict = dict(zip(items_set, range(len(items_set))))
    matrix = np.zeros(df_items.shape)
    
    for i in range(df.shape[0]):
        for item in df['items'][i]:
            #set matrix to boolean for item precence in basket:
            if item[1] not in stopwords:
                if item[2] > 0:
                    value = 1
                elif item[2] == 0:
                    value = 0
                else:
                    value = -1
                matrix[i,col_index_dict[ item[1] ]] = matrix[i,col_index_dict[ item[1] ]] + value
    sparse_matrix = sparse.csr_matrix(matrix)
    sparse_matrix = (sparse_matrix > 0).astype(int)
    
    return sparse_matrix

In [18]:
def fit_NMF(sparse_matrix,n_components=10,max_iter=250):
    from sklearn.decomposition import NMF
    model = NMF(n_components=n_components,max_iter=max_iter)
    W = model.fit_transform(sparse_matrix)
    H=model.components_
    model_iter = model.n_iter_
    return model,W,H,model_iter


from sklearn.decomposition import LatentDirichletAllocation as LDA
LDA_model = LDA(n_components=2,max_iter=10)
LDA_W = model.fit_transform(sparse_matrix)
LDA_H = model.components_


In [31]:
def print_top_items(month,model, feature_names, n_top_words):
    topic_dict = defaultdict()
    topics =[]
    for topic_idx, topic in enumerate(model.components_):
        print("Month %d, Topic #%d:" %(month,topic_idx))
        #topic_string=(" ".join([feature_names[i]for i in topic.argsort()[:-n_top_words - 1:-1]]))
        topic_string=[feature_names[i]for i in topic.argsort()[:-n_top_words - 1:-1]]
        topic_dict[topic_idx]=topic_string
        '''topics.append(topic_string)
        print(topic_string)
        #print()
        wordcloud = WordCloud(max_font_size=500, max_words=1000, background_color="white").generate(str(topic_string).replace(" ", "_").replace("'",""))

        # Display the generated image:
        #plt.figure(1,figsize=(10,10))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        #plt.savefig('topic%d.png'%topic_idx)
        #plt.close()
        wordcloud.to_file('./img/2018/%d/month.topic%d.png'%(month,topic_idx))
        print('<img src="./img/2018/%d/month.topic%d.png">'%(month,topic_idx))'''
    return topic_dict
#topics = print_top_items(model,list(items_set),25)
#print("stopwords:",stopwords,"number of topics:",len(topics))

In [32]:
many_topics_dict=defaultdict()
for month in range(1,13):
    print(month)
    print("df time")
    df = df_big[(df_big['month']==month)]
    print("get items")
    items_set,stopwords,item_dict, basket_counts = get_items(df,most_common=5,least_common=5)
    print("build matrix")
    sparse_matrix = build_matrix(df,items_set,stopwords,basket_counts)
    print("fit NMF")
    model,W,H,model_iter = fit_NMF(sparse_matrix,n_components=10,max_iter=250)
    print("print top items")
    topics = print_top_items(month,model,list(items_set),25)
    many_topics_dict[month] = topics

1
df time
get items


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'list'>
build matrix
fit NMF
print top items
Month 1, Topic #0:
Month 1, Topic #1:
Month 1, Topic #2:
Month 1, Topic #3:
Month 1, Topic #4:
Month 1, Topic #5:
Month 1, Topic #6:
Month 1, Topic #7:
Month 1, Topic #8:
Month 1, Topic #9:
2
df time
get items


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'list'>
build matrix
fit NMF
print top items
Month 2, Topic #0:
Month 2, Topic #1:
Month 2, Topic #2:
Month 2, Topic #3:
Month 2, Topic #4:
Month 2, Topic #5:
Month 2, Topic #6:
Month 2, Topic #7:
Month 2, Topic #8:
Month 2, Topic #9:
3
df time
get items


...

<class 'list'>
build matrix
fit NMF
print top items
Month 10, Topic #0:
Month 10, Topic #1:
Month 10, Topic #2:
Month 10, Topic #3:
Month 10, Topic #4:
Month 10, Topic #5:
Month 10, Topic #6:
Month 10, Topic #7:
Month 10, Topic #8:
Month 10, Topic #9:
11
df time
get items


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'list'>
build matrix
fit NMF
print top items
Month 11, Topic #0:
Month 11, Topic #1:
Month 11, Topic #2:
Month 11, Topic #3:
Month 11, Topic #4:
Month 11, Topic #5:
Month 11, Topic #6:
Month 11, Topic #7:
Month 11, Topic #8:
Month 11, Topic #9:
12
df time
get items


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'list'>
build matrix
fit NMF
print top items
Month 12, Topic #0:
Month 12, Topic #1:
Month 12, Topic #2:
Month 12, Topic #3:
Month 12, Topic #4:
Month 12, Topic #5:
Month 12, Topic #6:
Month 12, Topic #7:
Month 12, Topic #8:
Month 12, Topic #9:


In [47]:
len(many_topics_dict)*len(many_topics_dict[len(many_topics_dict)])

120

In [130]:
topic_matrix=np.zeros([len(many_topics_dict)*len(many_topics_dict[len(many_topics_dict)]),len(many_topics_dict)*len(many_topics_dict[len(many_topics_dict)])])

In [150]:
topics = []
topic_score=0

for month in many_topics_dict:
    #for topics in many:
    print ('month',month)
    column = 0
    for topic in many_topics_dict[month]:
        print ('month',month)
        print ("topic:",topic)
        #count+=1
        #print (count)
        topics.append(many_topics_dict[month][topic])
        
            
        for topic2 in many_topics_dict[month]:
            for item in many_topics_dict[month][topic]:
                if item in many_topics_dict[month][topic2]:
                    topic_score += 1
                topic_matrix[(month-1)+topic2,(month-1)+(topic*10)+topic2]=topic_score
                print((month-1)+topic,column)
                column+=1
            #print (item)
            #for item2 in many_topics_dict[many][topics]:
                

month 1
month 1
topic: 0
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180

9 2275
9 2276
9 2277
9 2278
9 2279
9 2280
9 2281
9 2282
9 2283
9 2284
9 2285
9 2286
9 2287
9 2288
9 2289
9 2290
9 2291
9 2292
9 2293
9 2294
9 2295
9 2296
9 2297
9 2298
9 2299
9 2300
9 2301
9 2302
9 2303
9 2304
9 2305
9 2306
9 2307
9 2308
9 2309
9 2310
9 2311
9 2312
9 2313
9 2314
9 2315
9 2316
9 2317
9 2318
9 2319
9 2320
9 2321
9 2322
9 2323
9 2324
9 2325
9 2326
9 2327
9 2328
9 2329
9 2330
9 2331
9 2332
9 2333
9 2334
9 2335
9 2336
9 2337
9 2338
9 2339
9 2340
9 2341
9 2342
9 2343
9 2344
9 2345
9 2346
9 2347
9 2348
9 2349
9 2350
9 2351
9 2352
9 2353
9 2354
9 2355
9 2356
9 2357
9 2358
9 2359
9 2360
9 2361
9 2362
9 2363
9 2364
9 2365
9 2366
9 2367
9 2368
9 2369
9 2370
9 2371
9 2372
9 2373
9 2374
9 2375
9 2376
9 2377
9 2378
9 2379
9 2380
9 2381
9 2382
9 2383
9 2384
9 2385
9 2386
9 2387
9 2388
9 2389
9 2390
9 2391
9 2392
9 2393
9 2394
9 2395
9 2396
9 2397
9 2398
9 2399
9 2400
9 2401
9 2402
9 2403
9 2404
9 2405
9 2406
9 2407
9 2408
9 2409
9 2410
9 2411
9 2412
9 2413
9 2414
9 2415
9 2416
9 2417

8 1962
8 1963
8 1964
8 1965
8 1966
8 1967
8 1968
8 1969
8 1970
8 1971
8 1972
8 1973
8 1974
8 1975
8 1976
8 1977
8 1978
8 1979
8 1980
8 1981
8 1982
8 1983
8 1984
8 1985
8 1986
8 1987
8 1988
8 1989
8 1990
8 1991
8 1992
8 1993
8 1994
8 1995
8 1996
8 1997
8 1998
8 1999
month 2
topic: 8
9 2000
9 2001
9 2002
9 2003
9 2004
9 2005
9 2006
9 2007
9 2008
9 2009
9 2010
9 2011
9 2012
9 2013
9 2014
9 2015
9 2016
9 2017
9 2018
9 2019
9 2020
9 2021
9 2022
9 2023
9 2024
9 2025
9 2026
9 2027
9 2028
9 2029
9 2030
9 2031
9 2032
9 2033
9 2034
9 2035
9 2036
9 2037
9 2038
9 2039
9 2040
9 2041
9 2042
9 2043
9 2044
9 2045
9 2046
9 2047
9 2048
9 2049
9 2050
9 2051
9 2052
9 2053
9 2054
9 2055
9 2056
9 2057
9 2058
9 2059
9 2060
9 2061
9 2062
9 2063
9 2064
9 2065
9 2066
9 2067
9 2068
9 2069
9 2070
9 2071
9 2072
9 2073
9 2074
9 2075
9 2076
9 2077
9 2078
9 2079
9 2080
9 2081
9 2082
9 2083
9 2084
9 2085
9 2086
9 2087
9 2088
9 2089
9 2090
9 2091
9 2092
9 2093
9 2094
9 2095
9 2096
9 2097
9 2098
9 2099
9 2100
9 2101
9 2

7 1392
7 1393
7 1394
7 1395
7 1396
7 1397
7 1398
7 1399
7 1400
7 1401
7 1402
7 1403
7 1404
7 1405
7 1406
7 1407
7 1408
7 1409
7 1410
7 1411
7 1412
7 1413
7 1414
7 1415
7 1416
7 1417
7 1418
7 1419
7 1420
7 1421
7 1422
7 1423
7 1424
7 1425
7 1426
7 1427
7 1428
7 1429
7 1430
7 1431
7 1432
7 1433
7 1434
7 1435
7 1436
7 1437
7 1438
7 1439
7 1440
7 1441
7 1442
7 1443
7 1444
7 1445
7 1446
7 1447
7 1448
7 1449
7 1450
7 1451
7 1452
7 1453
7 1454
7 1455
7 1456
7 1457
7 1458
7 1459
7 1460
7 1461
7 1462
7 1463
7 1464
7 1465
7 1466
7 1467
7 1468
7 1469
7 1470
7 1471
7 1472
7 1473
7 1474
7 1475
7 1476
7 1477
7 1478
7 1479
7 1480
7 1481
7 1482
7 1483
7 1484
7 1485
7 1486
7 1487
7 1488
7 1489
7 1490
7 1491
7 1492
7 1493
7 1494
7 1495
7 1496
7 1497
7 1498
7 1499
month 3
topic: 6
8 1500
8 1501
8 1502
8 1503
8 1504
8 1505
8 1506
8 1507
8 1508
8 1509
8 1510
8 1511
8 1512
8 1513
8 1514
8 1515
8 1516
8 1517
8 1518
8 1519
8 1520
8 1521
8 1522
8 1523
8 1524
8 1525
8 1526
8 1527
8 1528
8 1529
8 1530
8 1531
8 1

3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
3 36
3 37
3 38
3 39
3 40
3 41
3 42
3 43
3 44
3 45
3 46
3 47
3 48
3 49
3 50
3 51
3 52
3 53
3 54
3 55
3 56
3 57
3 58
3 59
3 60
3 61
3 62
3 63
3 64
3 65
3 66
3 67
3 68
3 69
3 70
3 71
3 72
3 73
3 74
3 75
3 76
3 77
3 78
3 79
3 80
3 81
3 82
3 83
3 84
3 85
3 86
3 87
3 88
3 89
3 90
3 91
3 92
3 93
3 94
3 95
3 96
3 97
3 98
3 99
3 100
3 101
3 102
3 103
3 104
3 105
3 106
3 107
3 108
3 109
3 110
3 111
3 112
3 113
3 114
3 115
3 116
3 117
3 118
3 119
3 120
3 121
3 122
3 123
3 124
3 125
3 126
3 127
3 128
3 129
3 130
3 131
3 132
3 133
3 134
3 135
3 136
3 137
3 138
3 139
3 140
3 141
3 142
3 143
3 144
3 145
3 146
3 147
3 148
3 149
3 150
3 151
3 152
3 153
3 154
3 155
3 156
3 157
3 158
3 159
3 160
3 161
3 162
3 163
3 164
3 165
3 166
3 167
3 168
3 169
3 170
3 171
3 172
3 173
3 174
3 175
3 176
3 177
3 178
3 179
3 180
3 181
3 182
3 183
3 184
3 185
3 

...

month 9
topic: 9
17 2250
17 2251
17 2252
17 2253
17 2254
17 2255
17 2256
17 2257
17 2258
17 2259
17 2260
17 2261
17 2262
17 2263
17 2264
17 2265
17 2266
17 2267
17 2268
17 2269
17 2270
17 2271
17 2272
17 2273
17 2274
17 2275
17 2276
17 2277
17 2278
17 2279
17 2280
17 2281
17 2282
17 2283
17 2284
17 2285
17 2286
17 2287
17 2288
17 2289
17 2290
17 2291
17 2292
17 2293
17 2294
17 2295
17 2296
17 2297
17 2298
17 2299
17 2300
17 2301
17 2302
17 2303
17 2304
17 2305
17 2306
17 2307
17 2308
17 2309
17 2310
17 2311
17 2312
17 2313
17 2314
17 2315
17 2316
17 2317
17 2318
17 2319
17 2320
17 2321
17 2322
17 2323
17 2324
17 2325
17 2326
17 2327
17 2328
17 2329
17 2330
17 2331
17 2332
17 2333
17 2334
17 2335
17 2336
17 2337
17 2338
17 2339
17 2340
17 2341
17 2342
17 2343
17 2344
17 2345
17 2346
17 2347
17 2348
17 2349
17 2350
17 2351
17 2352
17 2353
17 2354
17 2355
17 2356
17 2357
17 2358
17 2359
17 2360
17 2361
17 2362
17 2363
17 2364
17 2365
17 2366
17 2367
17 2368
17 2369
17 2370
17 2371
17 2372

16 1829
16 1830
16 1831
16 1832
16 1833
16 1834
16 1835
16 1836
16 1837
16 1838
16 1839
16 1840
16 1841
16 1842
16 1843
16 1844
16 1845
16 1846
16 1847
16 1848
16 1849
16 1850
16 1851
16 1852
16 1853
16 1854
16 1855
16 1856
16 1857
16 1858
16 1859
16 1860
16 1861
16 1862
16 1863
16 1864
16 1865
16 1866
16 1867
16 1868
16 1869
16 1870
16 1871
16 1872
16 1873
16 1874
16 1875
16 1876
16 1877
16 1878
16 1879
16 1880
16 1881
16 1882
16 1883
16 1884
16 1885
16 1886
16 1887
16 1888
16 1889
16 1890
16 1891
16 1892
16 1893
16 1894
16 1895
16 1896
16 1897
16 1898
16 1899
16 1900
16 1901
16 1902
16 1903
16 1904
16 1905
16 1906
16 1907
16 1908
16 1909
16 1910
16 1911
16 1912
16 1913
16 1914
16 1915
16 1916
16 1917
16 1918
16 1919
16 1920
16 1921
16 1922
16 1923
16 1924
16 1925
16 1926
16 1927
16 1928
16 1929
16 1930
16 1931
16 1932
16 1933
16 1934
16 1935
16 1936
16 1937
16 1938
16 1939
16 1940
16 1941
16 1942
16 1943
16 1944
16 1945
16 1946
16 1947
16 1948
16 1949
16 1950
16 1951
16 1952
16 1953


15 1358
15 1359
15 1360
15 1361
15 1362
15 1363
15 1364
15 1365
15 1366
15 1367
15 1368
15 1369
15 1370
15 1371
15 1372
15 1373
15 1374
15 1375
15 1376
15 1377
15 1378
15 1379
15 1380
15 1381
15 1382
15 1383
15 1384
15 1385
15 1386
15 1387
15 1388
15 1389
15 1390
15 1391
15 1392
15 1393
15 1394
15 1395
15 1396
15 1397
15 1398
15 1399
15 1400
15 1401
15 1402
15 1403
15 1404
15 1405
15 1406
15 1407
15 1408
15 1409
15 1410
15 1411
15 1412
15 1413
15 1414
15 1415
15 1416
15 1417
15 1418
15 1419
15 1420
15 1421
15 1422
15 1423
15 1424
15 1425
15 1426
15 1427
15 1428
15 1429
15 1430
15 1431
15 1432
15 1433
15 1434
15 1435
15 1436
15 1437
15 1438
15 1439
15 1440
15 1441
15 1442
15 1443
15 1444
15 1445
15 1446
15 1447
15 1448
15 1449
15 1450
15 1451
15 1452
15 1453
15 1454
15 1455
15 1456
15 1457
15 1458
15 1459
15 1460
15 1461
15 1462
15 1463
15 1464
15 1465
15 1466
15 1467
15 1468
15 1469
15 1470
15 1471
15 1472
15 1473
15 1474
15 1475
15 1476
15 1477
15 1478
15 1479
15 1480
15 1481
15 1482


11 12
11 13
11 14
11 15
11 16
11 17
11 18
11 19
11 20
11 21
11 22
11 23
11 24
11 25
11 26
11 27
11 28
11 29
11 30
11 31
11 32
11 33
11 34
11 35
11 36
11 37
11 38
11 39
11 40
11 41
11 42
11 43
11 44
11 45
11 46
11 47
11 48
11 49
11 50
11 51
11 52
11 53
11 54
11 55
11 56
11 57
11 58
11 59
11 60
11 61
11 62
11 63
11 64
11 65
11 66
11 67
11 68
11 69
11 70
11 71
11 72
11 73
11 74
11 75
11 76
11 77
11 78
11 79
11 80
11 81
11 82
11 83
11 84
11 85
11 86
11 87
11 88
11 89
11 90
11 91
11 92
11 93
11 94
11 95
11 96
11 97
11 98
11 99
11 100
11 101
11 102
11 103
11 104
11 105
11 106
11 107
11 108
11 109
11 110
11 111
11 112
11 113
11 114
11 115
11 116
11 117
11 118
11 119
11 120
11 121
11 122
11 123
11 124
11 125
11 126
11 127
11 128
11 129
11 130
11 131
11 132
11 133
11 134
11 135
11 136
11 137
11 138
11 139
11 140
11 141
11 142
11 143
11 144
11 145
11 146
11 147
11 148
11 149
11 150
11 151
11 152
11 153
11 154
11 155
11 156
11 157
11 158
11 159
11 160
11 161
11 162
11 163
11 164
11 165
11 166
11 

18 1808
18 1809
18 1810
18 1811
18 1812
18 1813
18 1814
18 1815
18 1816
18 1817
18 1818
18 1819
18 1820
18 1821
18 1822
18 1823
18 1824
18 1825
18 1826
18 1827
18 1828
18 1829
18 1830
18 1831
18 1832
18 1833
18 1834
18 1835
18 1836
18 1837
18 1838
18 1839
18 1840
18 1841
18 1842
18 1843
18 1844
18 1845
18 1846
18 1847
18 1848
18 1849
18 1850
18 1851
18 1852
18 1853
18 1854
18 1855
18 1856
18 1857
18 1858
18 1859
18 1860
18 1861
18 1862
18 1863
18 1864
18 1865
18 1866
18 1867
18 1868
18 1869
18 1870
18 1871
18 1872
18 1873
18 1874
18 1875
18 1876
18 1877
18 1878
18 1879
18 1880
18 1881
18 1882
18 1883
18 1884
18 1885
18 1886
18 1887
18 1888
18 1889
18 1890
18 1891
18 1892
18 1893
18 1894
18 1895
18 1896
18 1897
18 1898
18 1899
18 1900
18 1901
18 1902
18 1903
18 1904
18 1905
18 1906
18 1907
18 1908
18 1909
18 1910
18 1911
18 1912
18 1913
18 1914
18 1915
18 1916
18 1917
18 1918
18 1919
18 1920
18 1921
18 1922
18 1923
18 1924
18 1925
18 1926
18 1927
18 1928
18 1929
18 1930
18 1931
18 1932


In [132]:
topic_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [79]:
for topic in topics:
    for item in topic:
        for topic2 in topics:
            if item in topic2:
                print (topic == topic2)

True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False


False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
True
False
False
False
False
False
False
False

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fal

...

True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False


False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
Fal

In [73]:
topics2=topics.copy()

In [74]:
topic_compare =np.zeros([len(topics),len(topics)])

for topic in topics:
    for item in topics[topic]:
        for topic2 in topics:
            #print (item)
            #print (type(topics[topic2]))
            if item in topics[topic2]:
                topic_compare[topic,topic2]+=1
            
print(topic_compare)


TypeError: list indices must be integers or slices, not list

In [ ]:
list_of_topics=[]
for topic in topics:
     list_of_topics.append(topics[topic])

In [ ]:
len(topics[0])

In [ ]:
ingredients =np.array((len(topics),len(topics[0])))
for row in topics:
    print('row',row)
    for i,item in enumerate(topics[row]):
        if item not in topics[row]:
            print ('i',i,item)
            
        if item not in topics[0]:
            print ('iI',i,item)
        #ingredients[row:i]=item
ingredients

In [ ]:
len(ing_set)

In [ ]:
ing_index_dict = dict(zip(ing_set, range(len(ing_set))))

In [ ]:
ingredients=np.array(ingredients)
ingredients=ingredients.reshape(5,55)

In [ ]:
ingredients.shape

In [ ]:
ing_matrix=np.zeros(ingredients.shape)

In [ ]:
for row,col in enumerate(ingredients):
    for i,c in enumerate(col):
        #print(row,i,c)
        ing_matrix[row,i]=ing_index_dict[c]
ing_matrix

In [ ]:
ing_array=ing_index_dict.values()

ing_array

In [ ]:
for i in ing_matrix:
    print(ing_array==i)

In [ ]:
for ing_index_dict

In [ ]:
def print_basket_contents(basket_id):
    basket=[]
    #print('+++++++++++++++++++++++next basket+++++++++++++++++')
    for items in df['items'][basket_id]:
        #for item in items:
        print("item:",items)
    #return basket
#print_basket_contents(1116)

In [ ]:
def print_top_baskets(model, W, n_top_words):
    baskets = []
    for topic_idx, w in enumerate(W.T):
        basket_string=str([df['items'][i] for i in w.argsort()[:-n_top_words - 1:-1]])
        count = 0
        for i in w.argsort()[:-n_top_words - 1:-1]:
            count+=1
            print("++++++++++++++++++Topic #%d, basket #%d+++++++++++++++++++"%(topic_idx,count))
            print_basket_contents(i)
        #print_basket_contents([i for i in w.argsort()[:-n_top_words - 1:-1]])
        baskets.append(basket_string)
        #print(basket_string)
        #print()
        '''wordcloud = WordCloud(max_font_size=500, max_words=1000, background_color="white").generate(basket_string.replace(" ", "_").replace("'",""))

        # Display the generated image:
        #plt.figure(1,figsize=(10,10))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
        #plt.savefig('topic%d.png'%topic_idx)
        #plt.close()
        wordcloud.to_file('topic%d.png'%topic_idx)'''
print_top_baskets(model,W,10)